In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

In [ ]:
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv("C:/Users/Wommack/Downloads/dementia_dataset.csv")
df.head()

DATA CLEANING AND EXPLORATORY DATA ANALYSIS

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(), annot= True, cmap= 'Reds')

In [ ]:
df['Subject ID'].unique()

In [ ]:
# Subject_ID is dropped because it is unifor across all instances.
df=df.drop('Subject ID', axis= 1)
df.head()

In [ ]:
# MRI_ID is dropped because it is unifor across all instances.
df['MRI ID'].unique()

In [ ]:
df=df.drop('MRI ID', axis= 1)
df.head()

In [ ]:
df['Group'].unique()

Label Encoding of 'Group' Feature

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
df['Group']= le.fit_transform(df['Group'])
df['Group'].unique()

In [ ]:
df['Visit'].unique()

In [ ]:
df['M/F'].unique()

Binary Encoding of df['M/F']

In [ ]:
df['M/F']= df['M/F'].apply(lambda x: 1 if x == 'M' else (0 if x == 'F' else None))

In [ ]:
df['M/F'].unique()

In [ ]:
df['Hand'].unique()

the Hand column is dropped because its instances are the same for all rows

In [ ]:
df= df.drop('Hand',axis=1)
df.head()

In [ ]:
df['Age'].unique()

In [ ]:
df['EDUC'].unique()

In [ ]:
df['SES'].unique()

In [ ]:
df['SES'].value_counts()

In [ ]:
np.mean(df['SES'])

Replacing the nan values with the mode=2, since the mean is almost 2.

In [ ]:
df['SES']= df['SES'].replace(np.nan, 2)
df['SES'].unique()

In [ ]:
#df['SES'].value_counts()

In [ ]:
df['MMSE'].unique()

In [ ]:
print(df['MMSE'].mode())#,"\n",
print(df['MMSE'].mean())#,"\n",
print(df['MMSE'].value_counts())

In [ ]:
df['MMSE'].mean()

In [ ]:
df['MMSE'].isnull().sum()

Replacing the nan values with the mode=30, since nan values are 2 and mode =30.

In [ ]:
df['MMSE']= df['MMSE'].replace(np.nan, 30)
df['MMSE'].unique()

In [ ]:
df['CDR'].unique()

In [ ]:
df['eTIV'].unique()

In [ ]:
df['nWBV'].unique()

In [ ]:
df['ASF'].unique()

In [ ]:
df.head()

In [ ]:
X= df.drop('Group', axis=1)
y=df['Group']

In [ ]:
print(X.shape, y.shape)

BASE MODEL

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.3, random_state= 48 )

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score,StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn import metrics
import random as rd
models = [("LR", LinearRegression()),
          ("Naive Bayes", GaussianNB()),
          ("KNN", KNeighborsClassifier()),
          ("DTC", DecisionTreeClassifier()),
          ("SGD", SGDClassifier()),
          ("Ada", AdaBoostClassifier()),
          ("RFC",RandomForestClassifier()),
          ("Extra:",ExtraTreesClassifier()),
          #("SVC", SVC()),#has no feature importances
          ("XGBoost:", xgb.XGBClassifier())
         ]

 

In [ ]:
def model_evaluation(X,y):
    X_train,X_test, y_train, y_test= train_test_split(X, y, test_size= 0.3, random_state= 48 )
    models = [#("LR", LinearRegression()),
        ("Naive Bayes", GaussianNB()),
          ("KNN", KNeighborsClassifier()),
          ("DTC", DecisionTreeClassifier()),
          ("SGD", SGDClassifier()),
          ("Ada", AdaBoostClassifier()),
          ("RFC",RandomForestClassifier()),
          ("Extra:",ExtraTreesClassifier()),
          ("XGB:", xgb.XGBClassifier())
              
              
         ]
    means = []
    stds = []

    for name, classifier in models:
        scores = []
        for _ in range(10):
            model = classifier
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            Accuracy=metrics.accuracy_score(y_test,y_pred)

            scores.append(Accuracy)

        means.append(np.mean(scores))
        stds.append(np.std(scores))
        
    Total = list(zip(means,stds))
    hsh = {}
    for j in range(len(models)):
        hsh[models[j][0]] = list(Total[j])

    data = pd.DataFrame(hsh)
    db=data.transpose()
    db.columns=['mean','std']
    
    return db


In [ ]:
a=model_evaluation(X, y)
print(a)

In [ ]:
model= RandomForestClassifier()
model.fit(X_train, y_train)




y_pred = model.predict(X_test)
feature_imp= pd.Series(model.feature_importances_, index= X.columns).sort_values(ascending=False)
feature_imp*100

In [ ]:
#Choosing Features with values above the mean
from matplotlib import pyplot
pyplot.bar([x for x in range(len(model.feature_importances_))], model.feature_importances_)
pyplot.show()

In [ ]:
#Choosing Features with values above the mean
from matplotlib import pyplot
pyplot.bar([x for x in range(len(feature_imp))], feature_imp)
pyplot.show()

Choosing Top Two Features of NONE

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

In [ ]:
#split the dataset into features and labels
X1=df[['CDR','MMSE']]

y1=y

X1= scaler.fit_transform(X1)
         
X_train1, X_test1, y_train1, y_test1= train_test_split(X1, y1, test_size=0.30, random_state=45)
 

In [ ]:
b=model_evaluation(df[['CDR','ASF', 'eTIV']], y)
print(b)

In [ ]:
model= AdaBoostClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
feature_imp= pd.Series(model.feature_importances_, index= X.columns).sort_values(ascending=False)
feature_imp=feature_imp*100
feature_imp

In [ ]:
#split the dataset into features and labels
X2=df[['CDR','ASF', 'eTIV']]

y2=y

X2= scaler.fit_transform(X2)
         
X_train2, X_test2, y_train2, y_test2= train_test_split(X2, y2, test_size=0.30, random_state=45)
 

In [ ]:
c=model_evaluation(X2, y)
print(c)

In [ ]:
d=model_evaluation( df.drop('Group', axis=1),df['Group'])
print(d)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

#apply SelectKBest class to extract top 10 best features
bestfeatures=SelectKBest(score_func=mutual_info_classif, k=10)

fit= bestfeatures.fit(X,y)

In [ ]:
dfscores =pd.DataFrame(fit.scores_)
dfcolumns=pd.DataFrame(X.columns)

In [ ]:
#concat two dataframes for better visualization
featureScores= pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns=['Attri', 'Score'] #naming the dataframe columns
featureScores

In [ ]:
print(featureScores.nlargest(10, 'Score')) #print 5 best features

FEATURES SELECTION WITH F_CLASSIF

In [ ]:
from sklearn.feature_selection import f_classif

#apply SelectKBest class to extract top 10 best features
bestfeatures=SelectKBest(score_func=f_classif, k=10)

fit_f= bestfeatures.fit(X,y)

In [ ]:
dfscores_f =pd.DataFrame(fit_f.scores_)
dfcolumns_f=pd.DataFrame(X.columns)

In [ ]:
#concat two dataframes for better visualization
featureScores_f= pd.concat([dfcolumns_f, dfscores_f], axis=1)
featureScores_f.columns=['Attri', 'Score'] #naming the dataframe columns
#featureScores_f

In [ ]:
print(featureScores_f.nlargest(5, 'Score')) #print 5 best features

FEATURE SELECTION USING RECURSIVE FEATURE SELECTION

In [ ]:
from sklearn.feature_selection import RFECV
#Init, fit
rfecv= RFECV(
    estimator= RandomForestClassifier(),
    min_features_to_select=1,
    step=5,
    n_jobs= -1,
    scoring="r2",
    cv=5,
)

rfecv_fit= rfecv.fit(X_train, y_train)

In [ ]:
X_train_rf=X_train.columns[rfecv.support_]
X_train_rf

In [ ]:
b=model_evaluation(df[['MR Delay', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF']], y)
print(b)


In [ ]:
model= RandomForestClassifier()
model.fit(X_train, y_train)

#plot graphe of feature importances for better visualization
feat_importances= pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
model= xgb.XGBClassifier()
model.fit(X_train, y_train)

#plot graphe of feature importances for better visualization
feat_importances= pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model= ExtraTreesClassifier()
model.fit(X_train, y_train)

#plot graphe of feature importances for better visualization
feat_importances= pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
#CONCLUSION
#CDR is the most important factor in predicting dementia.
#Visit is the least import factor in predicring dementia according to Randpm classifier
#ASF is the least import factor in predicring dementia according to XGBoost
#SES is the least import factor in predicring dementia according to ExtraTreesClassifier 

